In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [5]:
input_ = []
s_time = time.time()
for i in range(6):
  input_.append(pd.read_excel('/gdrive/My Drive/input_'+str(i+1)+'.xlsx'))
print('Elapsed :', time.time() - s_time)

Elapsed : 46.92055654525757


In [6]:
# 전처리 1 : (연월종목 X 입력변수) 의 2차원 Dataframe 생성

# 6개 각각의 엑셀 파일을 2차원 dataframe으로 정리한 뒤 6개를 1개로 병합
new_input = []
for input_data in input_:
  for i in range(1,64): # 날짜 -> string
    input_data['Code'][i] = input_data['Code'][i].strftime('%Y%m')[2:]

  # 날짜 리스트
  date = input_data['Code'].tolist()[1:]

  num_input = 28 # 입력 변수 갯수
  num_stock = (input_data.shape[1] - 1)//num_input # 한 엑셀 파일당 종목 갯수
  num_date = len(date) # 기간 갯수

  # 종목별 데이터 쪼개기
  input_data_list = []
  for i in range(num_stock):
    input_data_list.append(input_data.iloc[:, num_input*i+1 : num_input*(i+1)+1])
  
  # 입력 변수 리스트
  input_list = input_data_list[0].loc[0,:].tolist()

  # 종목 당 (연월종목 X 입력변수) Dataframe 구성
  for i in range(num_stock):
    # 종목 코드 추출
    stock_code = input_data_list[i].columns[0]

    # 날짜 + 종목의 새 Index 구성
    new_date = [] 
    for t in date:
      new_date.append(t + stock_code)

    # Column 설정
    input_data_list[i].columns = input_list
    input_data_list[i].drop([0], inplace=True)

    # Index 설정
    input_data_list[i].index = new_date
  
  # 종목 Dataframe 결합
  new_input.append(pd.concat(input_data_list))

# 엑셀 별 Dataframe 결합
all_input = pd.concat(new_input)
all_input.to_excel('/gdrive/My Drive/all_input.xlsx')
all_input

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
# 전처리 2 : Dataset X, y 분리
# 주식 종목 당 X(t) <-> y(t+1) 이 연결되도록 한다.
# -> X : 가장 최근 값 삭제 / y : 가장 과거 값 삭제

num_date = 63 # 연월 종류 수
num_stock = all_input.shape[0]//num_date # 종목 종류 수

# y (Label) 분리
all_label = all_input[['월간수익률', '60일변동성(표준편차)']] 

# 종목 별 X 가장 최근 값 제거
input_idx_del = []
for i in range(num_stock):
  input_idx_del.append(all_input.index[num_date*(i+1) - 1])
all_input = all_input.drop(input_idx_del)

# 종목 별 y 가장 과거 값 제거
label_idx_del = []
for i in range(num_stock):
  label_idx_del.append(all_label.index[num_date * i])
all_label = all_label.drop(label_idx_del)

# X, y 결합
all_input['월간수익률'] = all_label['월간수익률'].tolist()
all_input['60일변동성(표준편차)'] = all_label['60일변동성(표준편차)'].tolist()
all_input

In [91]:
# 전처리 3 : 결측 값 제거 (y 값 NaN 시 제거)
# y 값 (월간수익률, 표준편차)가 NaN 인 경우가 적지만 
# 다른 값으로 채워넣기엔 심각한 성능 저하가 예측되기에 제거한다.
all_input = all_input[pd.notnull(all_input['월간수익률'])]
all_input = all_input[pd.notnull(all_input['60일변동성(표준편차)'])]
all_input

,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,비지배주주지분,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,8330.0,218978.0,12.48,0.007854,0.029854,0.00,0.72,6.36,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1505A000020,9370.0,332579.0,-3.20,0.010915,0.033701,0.00,0.40,7.00,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1506A000020,9070.0,295146.0,-10.03,0.008875,0.040963,0.00,0.88,8.21,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1507A000020,8160.0,596820.0,-12.99,0.017638,0.036095,0.00,1.39,8.54,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1508A000020,7100.0,130112.0,-1.13,0.004658,0.031218,0.02,2.45,8.56,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001A950180,16500.0,65335.0,-7.88,0.003102,0.018127,0.00,1.24,81.11,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002A950180,15200.0,32966.0,-13.16,0.001565,0.022581,0.00,3.25,81.52,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003A950180,13200.0,41955.0,13.64,0.001814,0.021570,0.00,1.68,81.69,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004A950180,15000.0,52057.0,12.33,0.002472,0.024074,0.00,0.02,81.93,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# 위의 과정 저장
all_input.to_excel('/gdrive/My Drive/all_input.xlsx')

In [97]:
# 위의 전처리 파일 존재 시 
s_time = time.time()
all_input = pd.read_excel('/gdrive/My Drive/all_input.xlsx')
print('Elapsed :', time.time() - s_time)

# 이상한 Column 제거
all_input.index = all_input['Unnamed: 0'].tolist()
all_input.drop(['Unnamed: 0'], axis='columns', inplace=True)

all_input

Elapsed : 31.15578842163086


,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,비지배주주지분,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,8330.0,218978.0,12.48,0.007854,0.029854,0.00,0.72,6.36,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1505A000020,9370.0,332579.0,-3.20,0.010915,0.033701,0.00,0.40,7.00,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1506A000020,9070.0,295146.0,-10.03,0.008875,0.040963,0.00,0.88,8.21,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1507A000020,8160.0,596820.0,-12.99,0.017638,0.036095,0.00,1.39,8.54,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
1508A000020,7100.0,130112.0,-1.13,0.004658,0.031218,0.02,2.45,8.56,61.99,1.350147e+08,1.821723e+08,87069287.63,27931470.0,2946737.22,NaN,17473114.34,19833109.34,-979732.31,885503.84,2234498.4,-2354408.39,12266194.33,1.266219e+08,96579379.29,91766405.62,31392013.0,4812973.67,5608652.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001A950180,16500.0,65335.0,-7.88,0.003102,0.018127,0.00,1.24,81.11,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002A950180,15200.0,32966.0,-13.16,0.001565,0.022581,0.00,3.25,81.52,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003A950180,13200.0,41955.0,13.64,0.001814,0.021570,0.00,1.68,81.69,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004A950180,15000.0,52057.0,12.33,0.002472,0.024074,0.00,0.02,81.93,64.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
